In [250]:
import numpy as np
import matplotlib.pyplot as plt
import pandas
import sys
import scipy
sys.path.append('/Users/ruby/EoR/pyuvdata')
from pyuvdata import UVData
import scipy.optimize

In [251]:
# Load data from pyuvsim simulation:
path = '/Users/ruby/EoR/compact_redundant_array_sim_May2020'
data_sim = UVData()
data_sim.read_uvh5('{}/square_grid_sim__results.uvh5'.format(path))

In [252]:
# Inflate data
print(np.shape(data_sim.baseline_array))
data_sim.inflate_by_redundancy()
print(np.shape(data_sim.baseline_array))

(61,)
(666,)


In [253]:
# Remove autos
data_sim.select(ant_str='cross')
# Use only XX polarizations
data_sim.select(polarizations=[-5])

In [254]:
# Convert baselines to have u>0
data_sim.conjugate_bls(convention='u>0', use_enu=False, uvw_tol=0.01)

In [255]:
baseline_groups, vec_bin_centers, lengths, conjugates = data_sim.get_redundancies(
    tol=0.1, use_antpos=False, include_conjugates=True, include_autos=True, conjugate_bls=False
)

In [256]:
np.shape(baseline_groups)[0]

60

In [257]:
print(np.shape(vec_bin_centers))
print(vec_bin_centers[1])

(60, 3)
[ 2.80000000e+01 -1.20564891e-10 -2.39678499e-10]


In [258]:
# Create the baseline covariance matrix
baseline_cov_array = np.diag(np.full(np.shape(baseline_groups)[0], 1.))
min_bl_length = 14.
tolerance = .01
for bl_1 in range(np.shape(baseline_groups)[0]):
    for bl_2 in [ind for ind in range(np.shape(baseline_groups)[0]) if ind != bl_1]:
        bl_separation_sq = (
            (vec_bin_centers[bl_1, 0]-vec_bin_centers[bl_2, 0])**2
            + (vec_bin_centers[bl_1, 1]-vec_bin_centers[bl_2, 1])**2
        )
        if (min_bl_length-tolerance)**2 <= bl_separation_sq <= (min_bl_length+tolerance)**2:
            baseline_cov_array[bl_1, bl_2] = 0.1617
        elif 2*(min_bl_length-tolerance)**2 <= bl_separation_sq <= 2*(min_bl_length+tolerance)**2:
            baseline_cov_array[bl_1, bl_2] = 0.0176

In [259]:
# Check that the covariance matrix is full-rank
print(N_baseline_groups)
print(np.linalg.matrix_rank(baseline_cov_array))

60
60


In [315]:
# Grab model visibilities
model_visibilities = np.zeros(np.shape(baseline_groups)[0], dtype=np.complex_)
for red_group in range(np.shape(baseline_groups)[0]):
    # Use first visiblity in each redundant group
    model_visibilities[red_group] = data_sim.data_array[
        np.where(data_sim.baseline_array == baseline_groups[red_group][0])[0], 0, 0, 0
    ]

In [316]:
# Initialize the fitted visibilities to the model visibility values
visibilities_initialize = np.copy(model_visibilities)
# Initialize the gains to 1
gains_initialize = np.full(data_sim.Nants_data, 1.+0.j)
# Expand the initialized values
x0 = np.concatenate((
    np.real(gains_initialize), np.imag(gains_initialize),
    np.real(visibilities_initialize), np.imag(visibilities_initialize)
))

In [317]:
# Create the A matrix
a_mat = np.zeros((np.shape(data_visibilities)[0], np.shape(baseline_groups)[0]))
for red_group in range(np.shape(baseline_groups)[0]):
    for baseline in baseline_groups[red_group]:
        a_mat[np.where(data_sim.baseline_array == baseline)[0], red_group] = 1

In [318]:
# Create gains expand matrices
gains_exp_mat_1 = np.zeros((np.shape(data_visibilities)[0], data_sim.Nants_data), dtype=np.complex_)
gains_exp_mat_2 = np.zeros((np.shape(data_visibilities)[0], data_sim.Nants_data), dtype=np.complex_)
for baseline in range(np.shape(data_visibilities)[0]):
    gains_exp_mat_1[baseline, data_sim.ant_1_array[baseline]] = 1
    gains_exp_mat_2[baseline, data_sim.ant_2_array[baseline]] = 1

In [319]:
def cost_function(
    x, 
    N_red_baselines, Nants, baseline_cov_array, model_visibilities, a_mat,
    gains_exp_mat, data_visibilities
):

    fitted_visibilities = x[-2*N_red_baselines:-N_red_baselines]+1j*x[-N_red_baselines:]
    gains = x[:Nants]+1j*x[Nants:2*Nants]
    
    vis_diff = fitted_visibilities-model_visibilities
    prior = np.abs(np.dot(np.matmul(np.conj(vis_diff), baseline_cov_array), vis_diff))
    
    fitted_visibilities_expanded = np.matmul(a_mat, fitted_visibilities)
    gains_expanded = np.matmul(gains_exp_mat_1, gains)*np.matmul(gains_exp_mat_2, np.conj(gains))
    prob = np.sum(np.abs(data_visibilities - gains_expanded*fitted_visibilities_expanded)**2)
    
    return prob+prior

In [320]:
a = cost_function(
    x0, 
    N_red_baselines, Nants, baseline_cov_array, model_visibilities, a_mat,
    gains_exp_mat, data_visibilities
)
print(a)

0.0006656440980293309


In [327]:
def add_visibility_noise(visibilities, stddev):
    noise = np.random.normal(0, stddev, np.shape(visibilities))
    noisy_visibilities = np.copy(visibilities)
    noisy_visibilities += noise
    return noisy_visibilities

In [328]:
# Create data visibilities
data_visibilities = add_visibility_noise(data_sim.data_array[:,0,0,0], 0.01)

# Run optimization
N_red_baselines = np.shape(baseline_groups)[0]
Nants = data_sim.Nants_data

result = scipy.optimize.minimize(
    cost_function, x0, 
    args=(
        N_red_baselines, Nants, baseline_cov_array, model_visibilities, a_mat, gains_exp_mat, data_visibilities
    ),
    method='Nelder-Mead', options={'maxiter':100000}
)
print(result.message)
gains_fit = result.x[:Nants]+1j*result.x[Nants:2*Nants]
print(gains_fit)

Optimization terminated successfully.
[1.00001906+8.79733132e-06j 0.99982489-2.82662811e-04j
 0.99980101+1.15840277e-06j 0.9999927 +8.12517952e-06j
 1.00002046-1.52946981e-04j 0.99996043+3.30389628e-04j
 1.00024225+2.35461269e-04j 0.99989903-4.85910103e-04j
 1.00002666+1.19702339e-04j 1.00021238-8.51138549e-05j
 1.00017224-3.23687253e-04j 0.9998535 +5.45185302e-04j
 1.00019567+2.99406951e-04j 1.0001212 +5.64294494e-06j
 0.99982597+2.23867040e-04j 0.99993733-2.55260584e-04j
 0.99969457-3.51223153e-05j 0.99987189-8.33558649e-05j
 0.99979009+2.98563342e-05j 0.99997414-2.16942306e-04j
 1.0003347 +1.60284709e-04j 1.00020141-1.02514748e-04j
 1.00003842-2.11507454e-04j 1.0000181 +1.90847580e-04j
 1.00019193-1.29592768e-04j 0.99999868-9.56584022e-07j
 0.99980203+2.90960691e-04j 1.00016107-2.26577555e-05j
 0.99985861+3.84657199e-04j 0.99979039-2.53866460e-05j
 0.99981343-1.77214448e-04j 0.99981322+7.84664547e-05j
 0.99985828+1.00959869e-04j 1.00007008+9.79097167e-06j
 0.99982165-8.91915074e-05j

In [324]:
print(result)

 final_simplex: (array([[ 0.9997887 ,  1.0000255 ,  0.99992596, ...,  1.61959632,
        -4.97414982, -1.04891889],
       [ 0.9997892 ,  1.00002859,  0.99992547, ...,  1.61964157,
        -4.97420697, -1.0488951 ],
       [ 0.99978951,  1.00002568,  0.99992465, ...,  1.61967063,
        -4.97423326, -1.04897495],
       ...,
       [ 0.99978758,  1.00002752,  0.99992592, ...,  1.61964713,
        -4.97416723, -1.04891465],
       [ 0.99979523,  1.00002597,  0.99992296, ...,  1.61969156,
        -4.97416474, -1.04901817],
       [ 0.99979038,  1.00002518,  0.99992484, ...,  1.61969479,
        -4.97425485, -1.04888268]]), array([0.05706853, 0.05706855, 0.05706858, 0.05706866, 0.05706867,
       0.0570687 , 0.05706875, 0.05706877, 0.05706882, 0.05706886,
       0.05706899, 0.057069  , 0.05706902, 0.05706903, 0.05706904,
       0.05706905, 0.05706905, 0.05706905, 0.05706908, 0.0570691 ,
       0.05706911, 0.05706911, 0.05706914, 0.05706914, 0.05706916,
       0.05706921, 0.05706924, 0.0

In [ ]:
# This returns all the baseline coordinates for a square array with antenna separations of 1
# For an NxN rectangular array, the number of redundant baseline groups is 2N^2-2N+1
# Excluding the autocorrelations, the number of redundant baseline groups is 2N^2-2N
array_side_length_ants = 6
N_baseline_groups = 2*array_side_length_ants**2-2*array_side_length_ants
baseline_group_coords = np.zeros((N_baseline_groups, 2))
bl_group_ind = 0
for uval in range(array_side_length_ants):
    if uval == 0:
        vrange = range(1, array_side_length_ants)
    else:
        vrange = range(-(array_side_length_ants-1), array_side_length_ants)
    for vval in vrange:
        baseline_group_coords[bl_group_ind, 0] = uval
        baseline_group_coords[bl_group_ind, 1] = vval
        bl_group_ind += 1